In [ ]:
# @title Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Intro to thought signatures with REST API

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures_rest.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fthinking%2Fintro_thought_signatures_rest.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/thinking/intro_thought_signatures_rest.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures_rest.ipynb">
      <img width="32px" src="https://raw.githubusercontent.com/primer/octicons/refs/heads/main/icons/mark-github-24.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<p>
<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures_rest.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures_rest.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures_rest.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures_rest.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures_rest.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>
</p>

| Authors |
| --- |
| [Eric Dong](https://github.com/gericdong) |

## Overview

A thought signature is an encrypted representation of the model's internal reasoning process for a given turn in a conversation. By passing this signature back to the model in subsequent requests, you provide it with the context of its previous thoughts, allowing it to build upon its reasoning and maintain a coherent line of inquiry.

This tutorial explores how to use the thought signatures feature of the Gemini API with cURL and the REST API.

## Getting Started

### Install required libraries

In [ ]:
%%capture

!sudo apt install -q jq

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
import os

# fmt: off
PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
# fmt: on
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "global")

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_REGION"] = LOCATION

## Use the Gemini 2.5 Flash model

In [ ]:
MODEL_ID = "gemini-2.5-flash"

api_host = "aiplatform.googleapis.com"
if LOCATION != "global":
    api_host = f"{LOCATION}-aiplatform.googleapis.com"

os.environ["API_ENDPOINT"] = (
    f"{api_host}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/publishers/google/models/{MODEL_ID}"
)
API_ENDPOINT = os.environ["API_ENDPOINT"]

## Use Thought Signatures

When thinking is enabled, the API response includes a `thought_signature` field containing an encrypted representation of the model's reasoning. When a function's execution result is sent back to the server, including the `thought_signature` allows the model to restore its previous thinking context, which will likely improve function calling performance.

Optionally, to enable thought summaries, include the `"thinking_config": { "include_thoughts": true }` object in your request body. 

## Example: Conditional Thermostat Control

In this scenario, a user wants to set a thermostat based on the current weather. The request is: "If it's too hot or too cold in London, set the thermostat to a comfortable temperature."

This requires the model to:

- Call a tool to get the weather in London.
- Use the returned weather information to decide if another tool needs to be called.
- Call the tool to set the thermostat if the condition is met.

### **Step 1**: First Turn - Get the Weather

We send the initial prompt to the model, along with the definitions of the tools it can use and the configuration to enable thinking. We expect it to call the `get_current_temperature` function and return a thought signature to maintain the context of the user's conditional request.

In [ ]:
%%bash

curl -X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://${API_ENDPOINT}:generateContent \
-d @- \
> response1.json 2>/dev/null <<EOF
{
    "contents": {
      "role": "user",
      "parts": { "text": "If it is too hot or too cold in London, set the temperature to a comfortable level. Make your own reasonable assumption for what comfortable means and do not ask for clarification." }
    },
    "tools": [
      {
        "function_declarations": [
          {
            "name": "get_current_temperature",
            "description": "Gets the current weather temperature for a given location.",
            "parameters": { "type": "object", "properties": {"location": {"type": "string"}}, "required": ["location"] }
          },
          {
            "name": "set_thermostat_temperature",
            "description": "Sets the thermostat to a desired temperature.",
            "parameters": { "type": "object", "properties": {"temperature": {"type": "integer"}}, "required": ["temperature"] }
          }
        ]
      }
    ],
    "generationConfig": {
      "thinking_config": {
        "include_thoughts": true
      },
    }
}
EOF

In [ ]:
%%bash

echo "--- Function Call ---"
jq -r '.candidates[0].content.parts[] | select(.functionCall) | .functionCall.name' response1.json

echo "--- Thought Signature ---"
jq -r '.candidates[0].content.parts[] | select(.functionCall) | .thoughtSignature' response1.json

echo "--- Thought Summary ---"
jq -r '.candidates[0].content.parts[] | select(.thought) | .text' response1.json

### **Step 2**: Second Turn - Set the Thermostat

Now, we send the model's previous response (containing the `functionCall` and the `thought_signature`) and the result from our first tool call back to the model. We will hardcode the values from the previous response for this demonstration.

⚠️ In this case, you modify the conversation history manually, instead of sending the complete previous response and want to benefit from thinking you must correctly handle the `thought_signature` included in the model's turn.

Follow these rules to ensure the model's context is preserved:

- Always send the `thought_signature` back to the model inside its original Part.
- Don't merge a Part containing a signature with one that does not. This breaks the positional context of the thought.
- Don't combine two Parts that both contain signatures, as the signature strings cannot be merged.

In [ ]:
jq_output = !jq -r '.candidates[0].content.parts[] | select(.functionCall) | .thoughtSignature' response1.json

os.environ["THOUGHT_SIGNATURE_1"] = jq_output[0]

In [ ]:
%%bash

curl -X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
"https://${API_ENDPOINT}:generateContent" \
-d @- \
> response2.json 2>/dev/null <<EOF
{
  "contents": [
      {
        "role": "user",
        "parts": { "text": "If it is too hot or too cold in London, set the temperature to a comfortable level. Make your own reasonable assumption for what comfortable means and do not ask for clarification." }
      },
      {
        "role": "model",
        "parts": [
          {
            "function_call": {
              "name": "get_current_temperature",
              "args": { "location": "London" }
            },
            "thought_signature": "${THOUGHT_SIGNATURE_1}"
          }
        ]
      },
      {
        "role": "tool",
        "parts": [ { "function_response": { "name": "get_current_temperature", "response": { "temperature": 30, "unit": "celsius" } } } ]
      }
    ],
    "tools": [
      {
        "function_declarations": [
          {
            "name": "get_current_temperature",
            "description": "Gets the current weather temperature for a given location.",
            "parameters": { "type": "object", "properties": {"location": {"type": "string"}}, "required": ["location"] }
          },
          {
            "name": "set_thermostat_temperature",
            "description": "Sets the thermostat to a desired temperature.",
            "parameters": { "type": "object", "properties": {"temperature": {"type": "integer"}}, "required": ["temperature"] }
          }
        ]
      }
    ],
    "generationConfig": {
      "thinking_config": {
        "include_thoughts": true
      },
    }
}
EOF

Using this context, the model will decide if it needs to call the `set_thermostat_temperature` function.

In [ ]:
%%bash

echo "--- Function Call ---"
jq -r '.candidates[0].content.parts[] | select(.functionCall) | .functionCall.name' response2.json

echo "--- Thought Signature ---"
jq -r '.candidates[0].content.parts[] | select(.functionCall) | .thoughtSignature' response2.json

echo "--- Thought Summary ---"
jq -r '.candidates[0].content.parts[] | select(.thought) | .text' response2.json

### **Step 3**: Generate a user-friendly response

Finally, we send the full conversation history, including the second function call and its result, back to the model to get a final, user-friendly text response.

In [ ]:
jq_output = !jq -r '.candidates[0].content.parts[] | select(.functionCall) | .thoughtSignature' response2.json

os.environ["THOUGHT_SIGNATURE_2"] = jq_output[0]

In [ ]:
%%bash
# NOTE: The history is copied from the previous steps.

curl -X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
"https://${API_ENDPOINT}:generateContent" \
-d @- \
> response3.json 2>/dev/null <<EOF
{
    "contents": [
      {
        "role": "user",
        "parts": { "text": "If it is too hot or too cold in London, set the temperature to a comfortable level. Make your own reasonable assumption for what 'comfortable' means and do not ask for clarification." }
      },
      {
        "role": "model",
        "parts": [
          {
            "function_call": {
              "name": "get_current_temperature",
              "args": { "location": "London" }
            },
            "thought_signature": "${THOUGHT_SIGNATURE_1}"
          }
        ]
      },
      {
        "role": "tool",
        "parts": [ { "function_response": { "name": "get_current_temperature", "response": { "temperature": 30, "unit": "celsius" } } } ]
      },
      {
        "role": "model",
        "parts": [
          {
            "function_call": {
              "name": "set_thermostat_temperature",
              "args": { "temperature": 21 }
            },
            "thought_signature": "${THOUGHT_SIGNATURE_2}"
          }
        ]
      },
      {
        "role": "tool",
        "parts": [ { "function_response": { "name": "set_thermostat_temperature", "response": { "status": "success" } } } ]
      }
    ],
    "tools": [
      {
        "function_declarations": [
          {
            "name": "get_current_temperature",
            "description": "Gets the current weather temperature for a given location.",
            "parameters": { "type": "object", "properties": {"location": {"type": "string"}}, "required": ["location"] }
          },
          {
            "name": "set_thermostat_temperature",
            "description": "Sets the thermostat to a desired temperature.",
            "parameters": { "type": "object", "properties": {"temperature": {"type": "integer"}}, "required": ["temperature"] }
          }
        ]
      }
    ]
}
EOF

echo "--- Final Text Response ---"
jq -r '.candidates[0].content.parts[0].text' response3.json

### **Cleanup**


In [ ]:
!rm response*.json

## Next Steps

Explores the thought signatures feature of the Gemini API with Gen AI SDK: [intro_thought_signatures.ipynb](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/thinking/intro_thought_signatures.ipynb)